In [1]:
import requests
import arcpy
import json
import zipfile

# 2. PRISM data

`PRISM_<var>_<stability>_<scale&version>_<time period>[_all][_<format>].<ext>`
    
https://prism.oregonstate.edu/documents/PRISM_downloads_FTP.pdf

https://ftp.prism.oregonstate.edu/normals/monthly/4km/ppt/

In [2]:
r = requests.get(
    'https://ftp.prism.oregonstate.edu/normals/monthly/4km/ppt/PRISM_ppt_30yr_normal_4kmM4_all_bil.zip'
)

with open(r'C:\Users\rozan012\Documents\ArcGIS\Projects\Lab2_1_2\PRISM.zip', 'wb') as f:
    f.write(r.content)

with zipfile.ZipFile(r'C:\Users\rozan012\Documents\ArcGIS\Projects\Lab2_1_2\PRISM.zip', 'r') as zip_ref:
        zip_ref.extractall('prism')

In [3]:
mosaic = arcpy.management.CreateMosaicDataset(
     in_workspace = r"C:\Users\rozan012\Documents\ArcGIS\Projects\Lab2_1_2\Lab2_1_2.gdb",
     in_mosaicdataset_name = "mosaic",
    coordinate_system = arcpy.SpatialReference(3857),
)

raster = arcpy.management.AddRastersToMosaicDataset(
    in_mosaic_dataset = mosaic,
    raster_type = 'Raster Dataset',
    input_path = r"C:\Users\rozan012\Documents\ArcGIS\Projects\Lab2_1_2\prism"
)

arcpy.management.CalculateField(
    in_table = r"mosaic\Footprint",
    field = "Variable",
    expression = '"Precip"',
    expression_type = "PYTHON3",
    code_block = '',
    field_type = 'TEXT',
)
arcpy.management.CalculateField(
    in_table = r"mosaic\Footprint",
    field = "Timestamp",
    expression = 'DateAdd(Date(1991, 0, 1), $feature.OBJECTID-1, "year")',
    expression_type = "Arcade",
    code_block = '',
    field_type = 'Date',
)

<Result 'mosaic\\Footprint'>

In [4]:
arcpy.management.BuildMultidimensionalInfo(
    in_mosaic_dataset = "mosaic",
    variable_field = "Variable",
    dimension_fields = "Timestamp",
)

<Result 'mosaic'>

In [5]:
multidim = arcpy.md.MakeMultidimensionalRasterLayer(
    in_multidimensional_raster = 'mosaic',
    out_multidimensional_raster_layer = r"C:\Users\rozan012\Documents\ArcGIS\Projects\Lab2_1_2\prism",
    variables = ['Precip'],
    dimension_def = 'ALL'
)

In [6]:
arcpy.stpm.CreateSpaceTimeCubeMDRasterLayer(
    in_md_raster = multidim,
    output_cube = r"C:\Users\rozan012\Documents\ArcGIS\Projects\Lab2_1_2\prism.nc",
)

<Result 'C:\\Users\\rozan012\\Documents\\ArcGIS\\Projects\\Lab2_1_2\\prism\\prism.nc'>